For the reformatting of the mutant data:

1 - Averages / standard deviation of growth rates of final OD600 / lag phase / GFP production etc. of technical replicates per experiments.

There are always 2 plates per experiment with the exact same content.

FOR DATA SET MUT1 - MUT3
Plate 1&2 = MUT.0001

Plate 3&4 = MUT.0002

Plate 5&6 = MUT.0003

Plate 7&8 = TS008

Plate 9&10 = TS071


Plate 11&12 = Blank medium, no cells


FOR DATA SET MUT5-MUT7

Plate 1&2 = MUT.0005

Plate 3&4 = MUT.0006

Plate 5&6 = MUT.0007

Plate 7&8 = TS008 (control strain 1)

Plate 9&10 = TS071 (control strain 2)

Plate 11&12 = Blank medium, no cells


2 - Averages / standard deviation of growth rates of final OD600 / lag phase / GFP production etc. of the three independent biological replicates executed at three different dates.

3 - In the end, I would like to have a matrix per strain with all averages and stand dev's of all the parameters you calculated. Then I can nicely plot the data for each mutant and make a comparison. I will already check the individual data sets today / tomorrow. 

In [1]:
import os
def getSubFolders(folderPath):
    subFolders = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and 'stats' in fn:
                pth = os.path.join(root, fn)
                if root in subFolders:
                    continue
                subFolders.append(pth)
    return subFolders

In [2]:
folder_input = '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/Mutant Data - Song/Mutant 1-3 + controls TS008 TS071'
lf = getSubFolders(folder_input)
folder_list = []
plateRule=[]
filter_out = ['Plate11','Plate12']
import os
g = dict()
for subFolder in lf:
    #print subFolder
    plate_name = os.path.split(subFolder)[1]
    if plate_name not in g.keys():
        g[plate_name] = []
    g[plate_name].append(subFolder)


In [3]:
def getEx_dict(l):
    experiement_dict =dict()
    for s_n in l:
        k_bydate = os.path.split(os.path.split(os.path.split(s_n)[0])[0])[1]
        if k_bydate not in experiement_dict.keys():
            experiement_dict[k_bydate] = []
        experiement_dict[k_bydate].append(s_n)
    return experiement_dict

group_files = dict()
for k in g.keys():    
    start =  k.index("Plate")    
    end =  k.index("_stats")
    i =  int(k[start+5:end])        
    if i%2 == 0:
        #print i
        j = i - 1        
        for m in g.keys():
            start =  m.index("Plate")    
            end =  m.index("_stats")
            i_m =  int(m[start+5:end])
            if i_m == j:
                print i_m, i
                group_files[(i_m,i)] = g[k]+g[m]

for k in group_files.keys():
    l = group_files[k]
    ex_byDate = getEx_dict(l)
    group_files[k] = ex_byDate            

7 8
9 10
5 6
3 4
1 2


In [4]:
def flatten_l(l0,l1):
    l0.extend(l1)
    return l0

def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]
    
def link_dict(dict1, dict2):    
    result = {key: flatten_l(as_list(dict1.get(key, 0)),as_list(dict2.get(key, 0)))
              for key in set(dict1) | set(dict2)}
    return result

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def mean_std_dict(dict0):
    import numpy as np
    result_m = {key+"_mean": np.mean(np.asarray(dict0.get(key,0))) for key in set(dict0)}
    result_std = {key+"_std": np.std(np.asarray(dict0.get(key,0))) for key in set(dict0)}
    return merge_two_dicts(result_m,result_std)

def getMeanStd(dic_growth):
    if len(dic_growth) > 1:
        dic_comb = dic_growth[0]
        for i in range(1,len(dic_growth)):                        
            dic_comb = link_dict(dic_growth[i], dic_comb)            
    else:
        print "only one value array, no mean and std"
        return dic_growth    
    result = mean_std_dict(dic_comb)
    return result



In [5]:
def getGrowthLag(stats_file):
    import csv
    m_growth = dict()
    m_lag = dict()
    with open(stats_file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                #print "row_0",row
                line_count += 1
            else:
                #print row
                line_count += 1
                m_growth[row[0]] = float(row[1])
                m_lag[row[0]] = float(row[2])
    return m_growth, m_lag


In [6]:
growth_rate = dict()
lag_phase = dict()

for k_group_ex in group_files.keys():
    print k_group_ex
    growth_rate[k_group_ex] = dict()
    lag_phase[k_group_ex] = dict()
    
    growth_rate[k_group_ex]['mixted_date'] = dict()
    lag_phase[k_group_ex]['mixted_date'] = dict()

    all_file = []
    dic_growth_all = []
    dic_lag_all = []
    
    stats_file_p = group_files[k_group_ex]
    for k_date in stats_file_p.keys():
        plate_file_date = stats_file_p[k_date] 
        print k_date        
        growth_rate[k_group_ex][k_date]= dict()
        lag_phase[k_group_ex][k_date]= dict()        
        
        dic_growth = []
        dic_lag = []
        for k_plate in plate_file_date:
            #print k_plate            
            all_file.append(k_plate)
            dic_g, dic_la = getGrowthLag(k_plate)
            dic_growth.append(dic_g)
            dic_lag.append(dic_la)
            
            dic_growth_all.append(dic_g)
            dic_lag_all.append(dic_la)
        
        growth_rate[k_group_ex][k_date] = getMeanStd(dic_growth)
        lag_phase[k_group_ex][k_date] = getMeanStd(dic_lag)
        
    growth_rate[k_group_ex]['mixted_date'] = getMeanStd(dic_growth_all)
    lag_phase[k_group_ex]['mixted_date'] = getMeanStd(dic_lag_all)
        
        

(9, 10)
9 Feb 2018
1 Feb 2018
20 Feb 2018
(5, 6)
9 Feb 2018
1 Feb 2018
20 Feb 2018
(3, 4)
9 Feb 2018
1 Feb 2018
20 Feb 2018
(1, 2)
9 Feb 2018
1 Feb 2018
20 Feb 2018
(7, 8)
9 Feb 2018
1 Feb 2018
20 Feb 2018


In [7]:
def sort_nicely(l):
    import re
    """ Sort the given list in the way that humans expect.
    """
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    l.sort(key=alphanum_key)
    return l



def get_all_plates(data_in):
    all_plates = dict()
    for g in sorted(data_in.keys()):
        out_file_name = "plate_"+str(g[0]) +"-plate_"+ str(g[1])
        date_all = []
        data_all = dict()    
        for date in sort_nicely(data_in[g].keys()):        
            #print date
            data_all[date]=dict()
            data = data_in[g][date]            
            for k_well in sort_nicely(data.keys()):                 
                #if k_well not in data_all.keys():
                data_all[date][k_well] = data[k_well]
                #else:
                    #data_all[date][k_well].extend(as_list(data[k_well]))            
        all_plates[out_file_name]=data_all
    return all_plates

all_plates_growth = get_all_plates(growth_rate)
all_plates_lagPhase = get_all_plates(lag_phase)

In [8]:
import csv
def writeToFile(f,all_plates,h):
    write_head = True
    for key in sorted(all_plates.keys()):
        if write_head:
            f.write("%s"%(key))
        else:
            f.write("%s\n"%(key))
        for k_date in sort_nicely(all_plates[key].keys()):
            string_to_write = "\t"
            head = "\t"
            for k_well in sort_nicely(all_plates[key][k_date].keys()):
                head = head + k_well+h + "\t"
                string_to_write = string_to_write + str(all_plates[key][k_date][k_well])+"\t"            
            if write_head:
                f.write("%s\n"%(head))
                write_head = False            
            f.write("%s"%(k_date))
            #f.write("%s\n"%(head))
            f.write("%s\n"%(string_to_write))


output_path = os.path.join(folder_input,'growth_rate_lag_phase.csv')
with open(output_path, 'w') as f:
    writeToFile(f,all_plates_growth,"_growth_rate")
    f.write("\n")
    writeToFile(f,all_plates_lagPhase,"_lag_phase")
    
            
    
            